<a href="https://colab.research.google.com/github/srdiegorodrigues/projeto/blob/main/ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#Importação das bibliotecas

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import io
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler

In [10]:
#Carregar arquivos a serem analisados
url = "/content/drive/MyDrive/MESTRADO/DADOS/OCORRENCIAS/"

df_2017 = pd.read_csv(io.StringIO(open(f'{url}datatran2017.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2018 = pd.read_csv(io.StringIO(open(f'{url}datatran2018.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2019 = pd.read_csv(io.StringIO(open(f'{url}datatran2019.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2020 = pd.read_csv(io.StringIO(open(f'{url}datatran2020.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2021 = pd.read_csv(io.StringIO(open(f'{url}datatran2021.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2022 = pd.read_csv(io.StringIO(open(f'{url}datatran2022.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)

df_2017 = df_2017.dropna()
df_2018 = df_2018.dropna()
df_2019 = df_2019.dropna()
df_2020 = df_2020.dropna()
df_2021 = df_2021.dropna()
df_2022 = df_2022.dropna()

dfs = [df_2017, df_2018, df_2019, df_2020, df_2021, df_2022]

<ipython-input-10-5849736c2be4>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2017 = pd.read_csv(io.StringIO(open(f'{url}datatran2017.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-10-5849736c2be4>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2018 = pd.read_csv(io.StringIO(open(f'{url}datatran2018.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-10-5849736c2be4>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2019 = pd.read_csv(io.StringIO(open(f'{url}datatran2019.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-10-5849736c2be4>:7: FutureWarning: The error_bad_lin

In [11]:
#carregar base de dados dos feriados nacionais
df_feriado = pd.read_csv("/content/drive/MyDrive/MESTRADO/DADOS/OCORRENCIAS/feriados_nacionais.csv",  encoding='ISO-8859-1', sep=";")

#arredondar o valor de km para cima ou para baixo, conforme o valor após o ponto
def arredondar_km(valor):
    valor_float = float(valor)  # Converter para float
    inteiro = int(valor_float)  # parte inteira do valor
    decimal = valor_float - inteiro  # parte decimal do valor

    # Se o valor decimal for maior ou igual a 0.5, arredonda para cima, caso contrário, arredonda para baixo
    if decimal >= 0.5:
        return inteiro + 1
    else:
        return inteiro

#converter o valor da feature br para int
def br_int(valor):
    valor_float = float(valor)
    valor_int = int(valor_float)
    return valor_int

dias_semana_para_int = {
        'segunda-feira': 0,
        'terça-feira': 1,
        'quarta-feira': 2,
        'quinta-feira': 3,
        'sexta-feira': 4,
        'sábado': 5,
        'domingo': 6
    }

def converter_dia_semana_para_int(dia):
    return dias_semana_para_int[dia]

for df in dfs:
    #alterar condições metereologicas para evitar outliers
    df['condicao_metereologica'] = df['condicao_metereologica'].str.replace('Ignorado', 'Céu Claro')
    df['condicao_metereologica'] = df['condicao_metereologica'].str.replace('Neve', 'Chuva')

    #substituir / por - na data_inversa
    df['data_inversa'] = df['data_inversa'].str.replace('/', '-')

    #criação da feature chuva para determinar se a pista estava molhada na hora do acidente
    for indice, item in df.iterrows():
        if item['condicao_metereologica'] in ['Chuva', 'Garoa/Chuvisco', 'Granizo']:
            df.at[indice, 'chuva'] = 1
        else:
            df.at[indice, 'chuva'] = 0

    #fragmentação da feature data_inversa em dia, mes e ano
    df[['dia', 'mes', 'ano']] = df['data_inversa'].apply(lambda x: pd.Series(x.split('-')).astype(int))

    # #Concatenar colunas data_inversa e horario e criando a feature data_hora
    df['data_hora'] = df.apply(lambda x: '%s-%s' % (x['data_inversa'],x['horario']), axis=1)
    #Criação da feature data_hora_int
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    # Convertendo as datas para timestamps Unix
    df['data_hora_int'] = df['data_hora'].astype(int) // 10**9

    #normalizar a feature data_hora_int
    scaler = MinMaxScaler()
    df['data_hora_int'] = scaler.fit_transform(df['data_hora_int'].values.reshape(-1, 1))

    #substituir a , por . na feature km
    df['km'] = df['km'].str.replace(',','.')
    df['km'] = df['km'].apply(arredondar_km)
    df['br'] = df['br'].apply(br_int)


    df['dia_semana_int'] =  df['dia_semana'].apply(converter_dia_semana_para_int)

    #Setar se dia da ocorrência do acidente era feriado ou não
    #Criando a feature feriado com base nas portarias do governo federal
    #Caso seja feriado, será atribuido o valor 1 para o registro
    #caso contrário, será atribuido o valor 0
    df['data_inversa'] = pd.to_datetime(df['data_inversa'])
    df_feriado['Data'] = pd.to_datetime(df_feriado['Data'])
    df_feriado['Data'] = df_feriado['Data'].sort_values(ascending=False).values
    # Criando uma nova coluna 'feriado' no DataFrame principal e inicializando com 0
    df['feriado'] = 0

    # Loop for para comparar as datas e atribuir 1 se forem iguais
    for index, row in df.iterrows():
        if row['data_inversa'] in df_feriado['Data'].values:
            df.at[index, 'feriado'] = 1

<ipython-input-11-74175e345fe5>:76: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['data_inversa'] = pd.to_datetime(df['data_inversa'])
<ipython-input-11-74175e345fe5>:77: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_feriado['Data'] = pd.to_datetime(df_feriado['Data'])


In [12]:
#features que serão usadas para execução dos modelos

# Selecionar as colunas categóricas
from sklearn.preprocessing import LabelEncoder, StandardScaler

cols_categoricas = ['uf','condicao_metereologica','delegacia','uop']

# Aplicar a codificação LabelEncoder em cada coluna categórica
label_encoder = LabelEncoder()
for df in dfs:
    for col in cols_categoricas:
        df[col] = label_encoder.fit_transform(df[col])

for df in dfs:
    df['regional_int'] = label_encoder.fit_transform(df['regional'])

colunas = ['km', 'uf', 'br', 'condicao_metereologica', 'regional','regional_int', 'delegacia', 'uop','chuva','dia','mes','ano','feriado', 'dia_semana_int']

In [13]:
#base de dados para algoritmos ANN, Gradiente Boosting Regressor
#treino anos 2017, 2018, 2019, 2020
#validação 2021
#teste 2022
treinamento2 = pd.concat([df_2017, df_2018, df_2019, df_2020], ignore_index=True)
treinamento2 = treinamento2[colunas]
teste2 = df_2021[colunas]
validacao2 = df_2022[colunas]

#dados para teste
treinamento2 = treinamento2[colunas]
sr_df = treinamento2[treinamento2['regional'] == 'SPRF-DF']
sr_df_br_020 = sr_df[sr_df['br'] == 20]
sr_df_br_040 = sr_df[sr_df['br'] == 40]
sr_sp = treinamento2[treinamento2['regional'] =='SPRF-SP']
sr_sp_br_101 = sr_sp[sr_sp['br'] == 101]
sr_sp_br_116 = sr_sp[sr_sp['br'] == 116]
sr_sc = treinamento2[treinamento2['regional'] =='SPRF-SC']
sr_sc_br_101 = sr_sc[sr_sc['br'] == 101]
sr_sc_br_116 = sr_sc[sr_sc['br'] == 116]

#dados para teste
sr_df_t = teste2[teste2['regional'] == 'SPRF-DF']
sr_df_br_020_t = sr_df_t[sr_df_t['br'] == 20]
sr_df_br_040_t = sr_df_t[sr_df_t['br'] == 40]
sr_sp_t = teste2[teste2['regional'] =='SPRF-SP']
sr_sp_br_101_t = sr_sp_t[sr_sp_t['br'] == 101]
sr_sp_br_116_t = sr_sp_t[sr_sp_t['br'] == 116]
sr_sc_t = teste2[teste2['regional'] =='SPRF-SC']
sr_sc_br_101_t = sr_sc_t[sr_sc_t['br'] == 101]
sr_sc_br_116_t = sr_sc_t[sr_sc_t['br'] == 116]

#dados para validação
sr_df_v = validacao2[validacao2['regional'] == 'SPRF-DF']
sr_df_br_020_v = sr_df_v[sr_df_v['br'] == 20]
sr_df_br_040_v = sr_df_v[sr_df_v['br'] == 40]
sr_sp_v = validacao2[validacao2['regional'] =='SPRF-SP']
sr_sp_br_101_v = sr_sp_v[sr_sp_v['br'] == 101]
sr_sp_br_116_v = sr_sp_v[sr_sp_v['br'] == 116]
sr_sc_v = validacao2[validacao2['regional'] =='SPRF-SC']
sr_sc_br_101_v = sr_sc_v[sr_sc_v['br'] == 101]
sr_sc_br_116_v = sr_sc_v[sr_sc_v['br'] == 116]

regionais = teste2['regional'].unique()

In [14]:
#bases de dados que serão utilizadas
df_treinamento = [sr_df, sr_df_br_020, sr_df_br_040, sr_sp, sr_sp_br_101,
                  sr_sp_br_116, sr_sc, sr_sc_br_101, sr_sc_br_116]
df_validacao =  [sr_df_v, sr_df_br_020_v, sr_df_br_040_v, sr_sp_v, sr_sp_br_101_v,
             sr_sp_br_116_v, sr_sc_v, sr_sc_br_101_v, sr_sc_br_116_v]
df_teste = [sr_df_t, sr_df_br_020_t, sr_df_br_040_t,sr_sp_t, sr_sp_br_101_t,
            sr_sp_br_116_t, sr_sc_t, sr_sc_br_101_t, sr_sc_br_116_t]

#MODELOS UTILIZANDO REDES NEURAIS ARTIFICIAIS



**Bases de dados que serão utilizadas**

O base de dados de treinamento é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias a partir do ano de 2017 até o ano de 2020


> df_treinamento = [sr_df, sr_df_br_020, sr_df_br_040, sr_sp, sr_sp_br_101,
                  sr_sp_br_116, sr_sc, sr_sc_br_101, sr_sc_br_116]



O base de dados de teste é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias no ano de 2021


>df_teste = [sr_df_t, sr_df_br_020_t, sr_df_br_040_t,sr_sp_t, sr_sp_br_101_t,
            sr_sp_br_116_t, sr_sc_t, sr_sc_br_101_t, sr_sc_br_116_t]



O base de dados de validação é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias no ano de 2022


> df_validacao =  [sr_df_v, sr_df_br_020_v, sr_df_br_040_v, sr_sp_v, sr_sp_br_101_v,sr_sp_br_116_v, sr_sc_v, sr_sc_br_101_v, sr_sc_br_116_v]






In [15]:
def avaliar_modelos(bases_treinamento, bases_teste, bases_validacao):
    for i, (base_treinamento, base_teste, base_validacao) in enumerate(zip(bases_treinamento, bases_teste, bases_validacao)):
        def imprimir_nome_base(df):
            base_name = [name for name, var in globals().items() if var is df][0]
            print('\n')
            print(f"Executando modelo com a base de treinamento: {base_name}")

        # Separar as características (X) e o alvo (y)
        imprimir_nome_base(base_treinamento)
        X_train = base_treinamento.drop(columns=['km','regional'])
        y_train = base_treinamento['km']

        X_test = base_teste.drop(columns=['km','regional'])
        y_test = base_teste['km']

        X_val = base_validacao.drop(columns=['km','regional'])
        y_val = base_validacao['km']

        # Normalizar os dados
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        X_val_scaled = scaler.transform(X_val)

        # Modelo de rede neural
        modelo = Sequential([
            Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
            Dropout(0.5),
            Dense(64, activation='relu'),
            Dropout(0.5),
            Dense(1)
        ])

        # Compilação do modelo
        modelo.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', 'mae'])

        # Treinamento do modelo
        modelo.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

        # Avaliação do modelo
        loss, mse, mae = modelo.evaluate(X_val_scaled, y_val)

        # Fazendo previsões
        y_pred = modelo.predict(X_val_scaled)

        # Calculando o erro médio quadrado (MSE)
        mse = mean_squared_error(y_val, y_pred)

        # Calculando o R² Score
        r2 = r2_score(y_val, y_pred)

        # Calculando o RMSE em quilômetros
        rmse_km = mean_squared_error(y_val, y_pred, squared=False)

        print(f"MSE: {mse}")
        print(f"R² Score: {r2}")
        print(f"RMSE (km): {rmse_km}")

# Exemplo de uso:
avaliar_modelos(df_treinamento, df_teste, df_validacao)




Executando modelo com a base de treinamento: sr_df
57/57 [==============================] - 0s 2ms/step
MSE: 409.5483599735744
R² Score: 0.7262722161558336
RMSE (km): 20.23730120281789


Executando modelo com a base de treinamento: sr_df_br_020
15/15 [==============================] - 0s 2ms/step
MSE: 1093.225136919385
R² Score: 0.6578529867783541
RMSE (km): 33.063955252198504


Executando modelo com a base de treinamento: sr_df_br_040
18/18 [==============================] - 0s 2ms/step
MSE: 289.8993903023438
R² Score: 0.6755470609815992
RMSE (km): 17.02643210723679


Executando modelo com a base de treinamento: sr_sp
137/137 [==============================] - 0s 2ms/step
MSE: 15314.003531923972
R² Score: 0.17479937344380936
RMSE (km): 123.749761744918


Executando modelo com a base de treinamento: sr_sp_br_101
6/6 [==============================] - 0s 4ms/step
MSE: 17879.64796784201
R² Score: -55.7568470869126
RMSE (km): 133.7148008555598


Executando modelo com a base de treinamen